# Day 9 - circular lists

Another fundamental datastructure puzzle today; after priority queues and stacks, today we need a circular list. We just have to rotate the circular list 1 step each time to add a marble; that way you essentially keep a reference to the *current* marble and can essentially ignore the remainder of the data structure.

For every 23rd round, add the marble to the current player's score, rotate the circular list 7 steps in the opposite direction and remove the marble there to also be added to the current player's score.

Once again we can reach for the [`collections.deque` object](https://docs.python.org/3/library/collections.html#collections.deque) here; it implements all the operations we need.

It may be that this is actually a fundamentaly mathematical problem, where we can just calculate the outcome, but I'm not yet aware of how that'd work. The implementation in Python, even with 10s of thousands of rounds, is plenty fast.

In [1]:
import re
from collections import deque
from typing import Tuple

def marble_game(playercount: int, marbles: int) -> int:
    """Play the elf marble game with a given number of players
    
    Returns the highest score.
    """
    # 'current' marble is last in the deque
    circle = deque([0])
    players = [0] * playercount
    player = 0
    for marble in range(1, marbles + 1):
        if marble % 23 == 0:
            # player is given this marble plus the one 7 steps clockwise
            circle.rotate(7)
            players[player % len(players)] += marble + circle.pop()
            # next 'current' is to the right of the removed element
            circle.rotate(-1)
        else:
            circle.rotate(-1)
            circle.append(marble)
        player += 1
    return max(players)

_parse_line = re.compile(r'(\d+) players; last marble is worth (\d+) points').search

def parse_input(line: str) -> Tuple[int, int]:
    match = _parse_line(line)
    assert match is not None
    pcount, marbles = map(int, match.groups())
    return pcount, marbles

In [2]:
tests = {
    '9 players; last marble is worth 25 points': 32,
    '10 players; last marble is worth 1618 points': 8317,
    '13 players; last marble is worth 7999 points': 146373,
    '17 players; last marble is worth 1104 points': 2764,
    '21 players; last marble is worth 6111 points': 54718,
    '30 players; last marble is worth 5807 points': 37305,
}
for testline, expected in tests.items():
    assert marble_game(*parse_input(testline)) == expected

In [3]:
import aocd

data = aocd.get_data(day=9, year=2018)
pcount, marbles = parse_input(data)

In [4]:
print('Part 1:', marble_game(pcount, marbles))

Part 1: 398371


## Scaling it up

> Amused by the speed of your answer, the Elves are curious:
>
> **What would the new winning Elf's score be if the number of the last marble were 100 times larger?**

Here it comes! Plenty fast in pure Python, eh? But even at 100x more iterations, Python with `deque` is plenty fast, so I've not searched for a mathematical approach here. It *could* be that the puzzle masters expected people to come up with less efficient array-based solutions and not use proper circular lists.

In [5]:
print('Part 2:', marble_game(pcount, marbles * 100))

Part 2: 3212830280


As it stands, part timings on my 2017-model Macbook Pro are in the 2 second range for part 2.

In [6]:
%timeit marble_game(pcount, marbles)
%timeit marble_game(pcount, marbles * 100)

22.1 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.24 s ± 6.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
